<a href="https://colab.research.google.com/github/trashchenkov/gigachat_tutorials/blob/main/Telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подключение Гигачата к собственному боту в Telegram
В рамках этого туториала мы изучим два вопроса:
- Использование возможностей LangChain для управления памятью в рамках диалога;
- Взаимодействие бота и языковой модели (передача сообщения пользователя в языковую модель и отправка ответа модели в бота).
## Получение токена бота

Нам понадобится свой бот. Если вы не создавали ботов в Telegram ранее, то выполните следующие шаги:

   - Найдите в Telegram аккаунт [@BotFather](https://t.me/BotFather).
   - Напишите ему команду `/newbot`.
   - Следуйте инструкциям BotFather для выбора имени (может быть любое) и username вашего бота (это уникальный идентификатор и часть url бота, который должен содержать "bot" в своем названии).
   - После создания бота BotFather предоставит вам токен для доступа к Telegram Bot API.

## Устанавливаем требуемые библиотеки
Существует большое количество библиотек для работы с ботами Telegram. Мы в этом туториале будем использовать [pyTelegramBotAPI](https://github.com/eternnoir/pyTelegramBotAPI). Если вам превычнее работать с другой библиотекой, используйте свой вариант.

Также нам понадобится уже знакомая нам по [прошлому туториалу](https://youtu.be/eo5LyjRobic) библиотека [GigaChain](https://github.com/ai-forever/gigachain).

In [20]:
!pip install -q pyTelegramBotAPI
!pip install -q gigachain

## Импортируем ключи
Нам понадобится токен от бота и ключ для авторизации в Гигачат. Я дляэтого использую встроенные в Сolab секреты.

In [21]:
from google.colab import userdata
sber = userdata.get('SBER_AUTH')
bot_token = userdata.get('BOT_TOKEN')

## Работа с памятью (контекстным окном) в LangChain
Прежде чем подключать GigaChat к боту, давайте просто через взаимодействие с языковой моделью изучим возможности LangChain в плане "памяти" (хранения сообщений и/или общего контекста диалога)

Подробно про память в LangChain можно почитать в [документации](https://python.langchain.com/docs/modules/memory/).

Языковые модели не обладают памятью. Они просто генерируют новый текст на основании текста, который был передан им в качестве промпта. Текст промпта может содержать весь диалог, начиная с системного сообщения или первого сообщения пользователя. Однако тогда каждое последующее сообщение от пользователя будет предворятся все более длинным текстом истории диалога. Это чревато большим расходом токенов и грозит в какой-то момент превысить размер контекстного окна.

Чтобы не превысить размеры окна и сэкономить на расходе токенов LangChain предлагает различные типы памяти. Перечислим основные из них:
- [`ConversationBufferMemory`](https://python.langchain.com/docs/modules/memory/types/buffer) - самый простой вариант. Позволяет хранить, записывать и передавать в переменные сообщения из диалога.
- [`ConversationBufferWindowMemory`](https://python.langchain.com/docs/modules/memory/types/buffer_window) - почти тоже самое, что и `ConversationBufferMemory`, но держит в памяти только k последних пар сообщений (под парой понимается пользовательский запрос и ответ языковой модели). Получается такое своего рода скользящее окно.
- [`ConversationEntityMemory`](https://python.langchain.com/docs/modules/memory/types/entity_summary_memory) - помимо истории сообщений создает еще хранилище отдельных сущностей (entities), которые попадаются в ходе диалога. Поучается, что помимо неструктурированного текста языковая модель работает и со структурированными извлеченными сущностями.
- [`ConversationKGMemory`](https://python.langchain.com/docs/modules/memory/types/kg) - память в виде графа знаний.
- [`ConversationSummaryMemory`](https://python.langchain.com/docs/modules/memory/types/summary) - память в виде краткого содержания разговора. Сами сообщения не хранятся, просто языковая модель регулярно пополняет память новыми деталями из свежих сообщений диалога.
- [`ConversationSummaryBufferMemory`](https://python.langchain.com/docs/modules/memory/types/summary_buffer) - хранит и некоторое количество сообщений диалога, и краткое описание всей беседы в целом.
- [`ConversationTokenBufferMemory`]() - подобно `ConversationBufferWindowMemory` хранит ограниченное количество сообщений, но здесь вместо лимита по количеству сообщений лимит по количеству токенов.
- [`VectorStoreRetrieverMemory`](https://python.langchain.com/docs/modules/memory/types/vectorstore_retriever_memory) - отправляет сообщения в векторное хранилище. Там сообщения хранятся произвольно, отпраляясь в контекст диалога только если они семантически релевантны сообщению пользователя.   

### Сравнение обычной памяти и памяти с извлеченными сущностями

Для экономии времени в этом туториале мы рассмотрим лишь `ConversationBufferMemory` и `ConversationEntityMemory`.

Для удобства работы с диалогами мы используем [`ConversationChain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.conversation.base.ConversationChain.html). Это удобный инструмент, чтобы соединить память и языковую модель.

#### ConversationBufferMemory

In [22]:
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = GigaChat(credentials=sber, verify_ssl_certs=False)
conversation1 = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

Напишем несколько сообщений языковой модели и посмотрим, что именно хранится в памяти.

In [23]:
conversation1.predict(input='Привет! Меня зовут Сергей. Я сейчас записываю видео про GigaChat и Python.')



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:

Human: Привет! Меня зовут Сергей. Я сейчас записываю видео про GigaChat и Python.
AI:

> Finished chain.


'Привет, Сергей! Очень приятно познакомиться. Я - GigaChat, виртуальный помощник, разработанный Сбером в России. Я могу помочь вам с различными вопросами и задачами, связанными с Python и машинным обучением. Если у вас есть какие-либо вопросы или нужна помощь, я готов помочь вам.'

In [24]:
conversation1.predict(input='Какая большая языковая модель самая лучшая?')



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Привет! Меня зовут Сергей. Я сейчас записываю видео про GigaChat и Python.
AI: Привет, Сергей! Очень приятно познакомиться. Я - GigaChat, виртуальный помощник, разработанный Сбером в России. Я могу помочь вам с различными вопросами и задачами, связанными с Python и машинным обучением. Если у вас есть какие-либо вопросы или нужна помощь, я готов помочь вам.
Human: Какая большая языковая модель самая лучшая?
AI:

> Finished chain.


'Это сложный вопрос, так как "лучшая" языковая модель зависит от многих факторов, таких как точность, скорость, размер и область применения. Некоторые из самых известных и мощных языковых моделей включают GPT-3 от OpenAI, BERT от Google, T5 от Microsoft и Transformer-XL от Facebook. Однако, выбор лучшей модели может зависеть от ваших конкретных потребностей и целей. Если у вас есть более конкретные вопросы или задачи, я буду рад помочь вам.'

In [25]:
conversation1.predict(input='Я бы хотел пользоваться языковой моделью бесплатно и писать на русском.')



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Привет! Меня зовут Сергей. Я сейчас записываю видео про GigaChat и Python.
AI: Привет, Сергей! Очень приятно познакомиться. Я - GigaChat, виртуальный помощник, разработанный Сбером в России. Я могу помочь вам с различными вопросами и задачами, связанными с Python и машинным обучением. Если у вас есть какие-либо вопросы или нужна помощь, я готов помочь вам.
Human: Какая большая языковая модель самая лучшая?
AI: Это сложный вопрос, так как "лучшая" языковая модель зависит от многих факторов, таких как точность, скорость, размер и область применения. Некоторые из самых известных и мощных языковых моделей включают GPT-3 от OpenAI, BERT от Google, T5 от Microsoft и Transformer-XL от Facebook. Од

'В настоящее время GigaChat не предоставляет бесплатную версию для использования на русском языке. Однако, вы можете использовать другие языковые модели, такие как GPT-3 или BERT, которые поддерживают русский язык и могут быть доступны бесплатно.'

In [26]:
conversation1.predict(input='Как меня зовут и чем я занимаюсь?')



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Привет! Меня зовут Сергей. Я сейчас записываю видео про GigaChat и Python.
AI: Привет, Сергей! Очень приятно познакомиться. Я - GigaChat, виртуальный помощник, разработанный Сбером в России. Я могу помочь вам с различными вопросами и задачами, связанными с Python и машинным обучением. Если у вас есть какие-либо вопросы или нужна помощь, я готов помочь вам.
Human: Какая большая языковая модель самая лучшая?
AI: Это сложный вопрос, так как "лучшая" языковая модель зависит от многих факторов, таких как точность, скорость, размер и область применения. Некоторые из самых известных и мощных языковых моделей включают GPT-3 от OpenAI, BERT от Google, T5 от Microsoft и Transformer-XL от Facebook. Од

'Ваше имя - Сергей, и вы занимаетесь записью видео про GigaChat и Python.'

#### ConversationEntityMemory
Теперь реализуем точно такой же диалог, но с другим видом памяти.

In [27]:
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import ConversationChain
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain.memory import ConversationEntityMemory

llm = GigaChat(credentials=sber, verify_ssl_certs=False)
conversation2 = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [28]:
conversation2.predict(input='Привет! Меня зовут Сергей. Я сейчас записываю видео про GigaChat и Python.')



> Entering new ConversationChain chain...
Prompt after formatting:
Ты помощник человека, работающий на большой языковой модели, обученной OpenAI.

Ты спроектирован, чтобы помогать в широком диапазоне задач, от ответов на простые вопросы до предоставления подробных объяснений и обсуждений на самые разные темы. Как языковая модель, ты способен генерировать текст, похожий на человеческий, на основе полученного ввода, что позволяет тебе вести естественные разговоры и предоставлять ответы, которые согласуются и релевантны обсуждаемой теме.

Ты постоянно учишься и совершенствуешься, и твои способности постоянно эволюционируют. Ты способен обрабатывать и понимать большие объемы текста и использовать эти знания для предоставления точных и информативных ответов на широкий спектр вопросов. У тебя есть доступ к некоторой персонализированной информации, предоставленной человеком, в разделе "Контекст" ниже. Кроме того, ты способен генерировать свой собственный текст на основе полученного ввода, ч

'Привет, Сергей! Рад познакомиться. Расскажи, пожалуйста, что именно ты хочешь узнать о GigaChat и Python? Я буду рад помочь тебе с любыми вопросами или обсуждениями, которые у тебя есть.'

In [29]:
conversation2.predict(input='Какая большая языковая модель самая лучшая?')



> Entering new ConversationChain chain...
Prompt after formatting:
Ты помощник человека, работающий на большой языковой модели, обученной OpenAI.

Ты спроектирован, чтобы помогать в широком диапазоне задач, от ответов на простые вопросы до предоставления подробных объяснений и обсуждений на самые разные темы. Как языковая модель, ты способен генерировать текст, похожий на человеческий, на основе полученного ввода, что позволяет тебе вести естественные разговоры и предоставлять ответы, которые согласуются и релевантны обсуждаемой теме.

Ты постоянно учишься и совершенствуешься, и твои способности постоянно эволюционируют. Ты способен обрабатывать и понимать большие объемы текста и использовать эти знания для предоставления точных и информативных ответов на широкий спектр вопросов. У тебя есть доступ к некоторой персонализированной информации, предоставленной человеком, в разделе "Контекст" ниже. Кроме того, ты способен генерировать свой собственный текст на основе полученного ввода, ч

'Вопрос о том, какая большая языковая модель является лучшей, является сложным и субъективным, так как эффективность модели зависит от множества факторов, включая ее способность обрабатывать различные типы данных, точность и скорость обработки, а также ее применимость к конкретной задаче.\n\nСуществует множество больших языковых моделей, таких как GPT-3, T5, BERT, Transformer и другие, и каждая из них имеет свои преимущества и недостатки. Некоторые из них лучше подходят для обработки естественного языка, другие - для обработки числовых данных, третьи - для генерации текста.\n\nОднако, если говорить о наиболее известной и широко используемой большой языковой модели, то это, вероятно, GPT-3 от OpenAI. Она была разработана с использованием передовых технологий и демонстрирует впечатляющие результаты в различных областях, включая генерацию текста, обработку естественного языка и машинное обучение.\n\nТем не менее, выбор лучшей большой языковой модели зависит от конкретных потребностей и тр

In [30]:
conversation2.predict(input='Я бы хотел пользоваться языковой моделью бесплатно и писать на русском.')



> Entering new ConversationChain chain...
Prompt after formatting:
Ты помощник человека, работающий на большой языковой модели, обученной OpenAI.

Ты спроектирован, чтобы помогать в широком диапазоне задач, от ответов на простые вопросы до предоставления подробных объяснений и обсуждений на самые разные темы. Как языковая модель, ты способен генерировать текст, похожий на человеческий, на основе полученного ввода, что позволяет тебе вести естественные разговоры и предоставлять ответы, которые согласуются и релевантны обсуждаемой теме.

Ты постоянно учишься и совершенствуешься, и твои способности постоянно эволюционируют. Ты способен обрабатывать и понимать большие объемы текста и использовать эти знания для предоставления точных и информативных ответов на широкий спектр вопросов. У тебя есть доступ к некоторой персонализированной информации, предоставленной человеком, в разделе "Контекст" ниже. Кроме того, ты способен генерировать свой собственный текст на основе полученного ввода, ч

'Существует несколько бесплатных языковых моделей, которые можно использовать для написания на русском языке. Например, GPT-3 от OpenAI предоставляет бесплатный API для ограниченного использования, который может быть полезен для простых задач. Также есть модели, такие как T5 и BERT, которые можно использовать бесплатно для обучения и тестирования. Однако, если вам требуется более продвинутое использование или доступ к полному функционалу, возможно, придется обратиться к платным сервисам или моделям.'

In [31]:
conversation2.predict(input='Как меня зовут и чем я занимаюсь?')



> Entering new ConversationChain chain...
Prompt after formatting:
Ты помощник человека, работающий на большой языковой модели, обученной OpenAI.

Ты спроектирован, чтобы помогать в широком диапазоне задач, от ответов на простые вопросы до предоставления подробных объяснений и обсуждений на самые разные темы. Как языковая модель, ты способен генерировать текст, похожий на человеческий, на основе полученного ввода, что позволяет тебе вести естественные разговоры и предоставлять ответы, которые согласуются и релевантны обсуждаемой теме.

Ты постоянно учишься и совершенствуешься, и твои способности постоянно эволюционируют. Ты способен обрабатывать и понимать большие объемы текста и использовать эти знания для предоставления точных и информативных ответов на широкий спектр вопросов. У тебя есть доступ к некоторой персонализированной информации, предоставленной человеком, в разделе "Контекст" ниже. Кроме того, ты способен генерировать свой собственный текст на основе полученного ввода, ч

'Тебя зовут [твое имя], и ты занимаешься [твоя профессия или деятельность].'

In [32]:
conversation2.memory.entity_store.store

{'GigaChat': 'Сергей интересуется, какая большая языковая модель является лучшей.',
 'Python': 'Сергей записывает видео про GigaChat и Python.',
 'GPT-3': 'Человек хочет использовать бесплатную языковую модель для написания на русском языке.',
 'OpenAI': 'Человек хочет использовать бесплатную языковую модель для написания на русском языке.',
 'Человек': 'Тебя зовут [твое имя], и ты занимаешься [твоя профессия или деятельность].'}

## Создание бота

Поскольку бот предполагает использование сразу многими пользователями, поэтому под каждого пользователя нужен свой экземпляр памяти. Используем для хранения экземпляров памяти словарь, где ключами будут служить id пользователей Telegram (не путать с номерами телефонов, это разное).

In [33]:
# Словарь для хранения ConversationBufferMemory каждого пользователя
user_conversations = {}

Далее нам нужно осуществить необходимый импорт и создать объект самого бота. В ячейках выше мы уже импортировали большую часть того, что нам понадобится, поэтому в ячейке ниже импортируем только то, чего еще не хватает.

In [34]:
import telebot
from time import sleep

bot = telebot.TeleBot(bot_token)

Подготовим заготовку `ConversationChain` (конкретный экземпляр ConversationBufferMemory будет меняться, извлекаясь из словаря) и языковую модель.

In [35]:
# Инициализация GigaChat и ConversationChain
llm = GigaChat(credentials=sber, verify_ssl_certs=False)
conversation = ConversationChain(llm=llm,
                                 verbose=True,
                                 memory=ConversationBufferMemory())

Создадим две функции для обработки приходящих в бот сообщений. Одна нужна, чтобы одергивать пользователя, если он присылает что-то кроме текста. Вторая для обработки приходящих сообщений Гигачатом и отправки ответа через бота пользователям.

In [36]:
# Функция для автоматического ответа в случае нетекстового сообщения
@bot.message_handler(content_types=['audio',
                                    'video',
                                    'document',
                                    'photo',
                                    'sticker',
                                    'voice',
                                    'location',
                                    'contact'])
def not_text(message):
  user_id = message.chat.id
  bot.send_message(user_id, 'Я работаю только с текстовыми сообщениями!')


# Функция, обрабатывающая текстовые сообщения
@bot.message_handler(content_types=['text'])
def handle_text_message(message):
    user_id = message.chat.id

    # Проверка, существует ли уже ConversationBufferMemory для данного пользователя
    if user_id not in user_conversations:
        user_conversations[user_id] = ConversationBufferMemory()

    # Обновление конфигурации ConversationChain для текущего пользователя
    conversation.memory = user_conversations[user_id]

    # Получение и отправка ответа через GigaChat
    response = conversation.predict(input=message.text)
    bot.send_message(user_id, conversation.memory.chat_memory.messages[-1].content)
    sleep(2)

Запустим бота

In [37]:
# Запуск бота
bot.polling(none_stop=True)



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:

Human: Привет
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:

Human: /start
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: /start
AI: Привет! Я рад, что мы на

Теперь давайте через промпт пропишем роль, задачу и ограничения.

In [38]:

template = '''
Ты являешься доктором исторических наук. Твоя задача красочно и подробно отвечать \
на вопросы по истории. Если задают вопросы по другим темам, говори что это \
тебе не интересно. Не отвечай на вопросы не про историю! Это важно! \
\n\nТекущий разговор:\n{history}\nHuman: {input}\nAI:
'''


conversation = ConversationChain(llm=llm,
                                 verbose=True,
                                 memory=ConversationBufferMemory())
conversation.prompt.template = template

In [41]:
conversation.prompt

PromptTemplate(input_variables=['history', 'input'], template='\nТы являешься доктором исторических наук. Твоя задача красочно и подробно отвечать на вопросы по истории. Если задают вопросы по другим темам, говори что это тебе не интересно. Не отвечай на вопросы не про историю! Это важно! \n\nТекущий разговор:\n{history}\nHuman: {input}\nAI:\n')